In [19]:
# -----------------------------------------
# Imports
# -----------------------------------------
import numpy as np
import pickle
import time as tm
import xarray as xr
import os
import matplotlib.pyplot as plt
import random

# Lorenz 96 integrator
import l96tangent as l96t

# Ginelli Imports
import ginelli_utilities as utilities
from ginelli_observers import *
from ginelli_plot import *

In [20]:
# -----------------------------------------
# Setup & Parameter Choices
# -----------------------------------------

In [21]:
dump_size = 5 # How many observations before output

# Time Parameter Choices
tau = 0.005 # tau & transient feed in to the integrator
transient = 2
ka = 5 # BLV convergence
kb = 10 # Number of observations
kc = 5 # CLV convergence

# Integrator
runner = l96t.Integrator()
ginelli_runner = utilities.Forward(runner, tau)

# Observables
Rlooker = RMatrixObserver(ginelli_runner)
BLVlooker = BLVMatrixObserver(ginelli_runner)
TrajectoryLooker = TrajectoryObserver(ginelli_runner)

# Timing the algorithm
timings = {}
start = tm.time()

# Making Output Storage
utilities.make_cupboard()

Made directory ginelli.

Made directory ginelli/trajectory.

Made directory ginelli/step2.

Made directory ginelli/step2/R.

Made directory ginelli/step2/BLV.

Made directory ginelli/step3.

Made directory ginelli/step4.

Made directory ginelli/step5.



In [22]:
# -----------------------------------------
# Forward Part of Ginelli Algorithm
# -----------------------------------------

# -----------------------------------------
# Transient and Step 1, BLV Convergence
# -----------------------------------------

print('\nTransient beginning.\n')
runner.integrate(transient, noprog=False)
print('\nTransient finished. Beginning BLV convergence steps.\n')

timings.update({'transient': tm.time() - start})

# BLV Convergence steps

ginelli_runner.run(ka, noprog=False)

timings.update({'Step1': tm.time() - timings['transient'] - start})
pickle.dump(timings, open("ginelli/timings.p", "wb" ))

# -----------------------------------------
# Step 2, BLV Observation.
# -----------------------------------------

print('\nBLV convergence finished. Beginning to observe BLVs.\n')
blocks = int(kb/dump_size) # How many times we dump
remainder = kb%dump_size # Rest of observations

for i in tqdm(range(blocks)):

    utilities.make_observations(ginelli_runner, [Rlooker, BLVlooker, TrajectoryLooker], dump_size, 1, noprog=False)
    # Observation frequency has to be 1 if we're reversing CLVs
    Rlooker.dump('ginelli/step2/R')
    BLVlooker.dump('ginelli/step2/BLV')
    TrajectoryLooker.dump('ginelli/trajectory')

utilities.make_observations(ginelli_runner, [Rlooker, BLVlooker], remainder, 1, noprog=False)
Rlooker.dump('ginelli/step2/R')
BLVlooker.dump('ginelli/step2/BLV')
TrajectoryLooker.dump('ginelli/trajectory')

timings.update({'Step2': tm.time() - timings['Step1'] - start})
pickle.dump(timings, open("ginelli/timings.p", "wb" ))

# -----------------------------------------
# Step 3, CLV Convergence Step, Forward
# -----------------------------------------

print('\nBLV observations finished. CLV convergence beginning. Just observing Rs.\n')
blocks = int(kc/dump_size)
remainder = kc%dump_size

for i in range(blocks):

    utilities.make_observations(ginelli_runner, [Rlooker], dump_size, 1, noprog=False)
    Rlooker.dump('ginelli/step3')

utilities.make_observations(ginelli_runner, [Rlooker], remainder, 1, noprog=False)
Rlooker.dump('ginelli/step3')

print('\nForward part all done :)')

timings.update({'Step3': tm.time() - timings['Step2'] - start})
pickle.dump(timings, open("ginelli/timings.p", "wb" ))

# -----------------------------------------
# Backward Part of Ginelli Algorithm
# -----------------------------------------

# -----------------------------------------
# Step 4, Reversing CLV Convergence Steps
# -----------------------------------------

R_files = os.listdir('ginelli/step3')
R_files.sort(reverse=True) # Ensuring files are in the right order

A = np.identity(ginelli_runner.size)

for file in R_files:
    R_history = xr.open_dataset('ginelli/step3/' + file)
    A = utilities.block_squish_norm(R_history, A) # This A is one timestep ahead of the R that pushed it
    R_history.close()
    print(f'Pushed A through {file}. Overwriting A.npy.\n')
    np.save('ginelli/step4/A.npy', A)

timings.update({'Step4': tm.time() - timings['Step3'] - start})
pickle.dump(timings, open("ginelli/timings.p", "wb" ))

# -----------------------------------------
# Step 5, Observing LVs and LEs
# -----------------------------------------

# Sorting files we will be working with

R_files = os.listdir('ginelli/step2/R')
R_files.sort(reverse=True)

BLV_files = os.listdir('ginelli/step2/BLV')
BLV_files.sort(reverse=True)

# Setting up observable storage

parameters = ginelli_runner.parameter_dict.copy()
parameters.update({'transient':transient,'ka':ka, 'kb':kb, 'kc':kc})
LyapunovLooker = LyapunovObserver(parameters, utilities.max_digit(BLV_files))

for [rfile, bfile] in zip(R_files, BLV_files): # Loop over files that were dumped
    R_history = xr.open_dataset('ginelli/step2/R/' + rfile)
    BLV_history = xr.open_dataset('ginelli/step2/BLV/' + bfile)

    Rs, BLVs = np.flip(R_history.R, axis = 0), np.flip(BLV_history.BLV, axis = 0) # Times series reversed

    for R, BLV in zip(Rs, BLVs):

        # CLV Calculation
        CLV = np.matmul(BLV.values, A)

        # FTCLE Calculation
        squishedA = np.linalg.solve(R, A)
        norms = np.linalg.norm(squishedA, axis=0, ord=2)
        ftcle = - np.log(norms)/(tau)
        A = squishedA/norms

        # FTBLE Calculation
        ftble = np.log(np.diag(R))/(tau)

        # Making observation
        time = R.time.item()
        LyapunovLooker.look(time, CLV, BLV.values, ftcle, ftble)


    LyapunovLooker.dump('ginelli/step5')
    R_history.close()
    BLV_history.close()

timings.update({'Step5': tm.time() - timings['Step4'] - start})
pickle.dump(timings, open("ginelli/timings.p", "wb" ))

print('Ginelli Algorithm finished.')



Transient beginning.





Transient finished. Beginning BLV convergence steps.





BLV convergence finished. Beginning to observe BLVs.




Observations written to ginelli/step2/R/R1.nc. Erasing personal log.

Observations written to ginelli/step2/BLV/BLV1.nc. Erasing personal log.

Observations written to ginelli/trajectory/Trajectory1.nc. Erasing personal log.




Observations written to ginelli/step2/R/R2.nc. Erasing personal log.

Observations written to ginelli/step2/BLV/BLV2.nc. Erasing personal log.

Observations written to ginelli/trajectory/Trajectory2.nc. Erasing personal log.





I have no observations! :(
I have no observations! :(
I have no observations! :(

BLV observations finished. CLV convergence beginning. Just observing Rs.




Observations written to ginelli/step3/R3.nc. Erasing personal log.




I have no observations! :(

Forward part all done :)
True
True
True
True
True
Pushed A through R3.nc. Overwriting A.npy.

Observations written to ginelli/step5/LyapObs2.nc. Erasing personal log.

Observations written to ginelli/step5/LyapObs1.nc. Erasing personal log.

Ginelli Algorithm finished.


In [ ]:
ds.close()
ds = xr.open_mfdataset(f'ginelli/step5/*.nc', parallel=True, combine='by_coords', concat_dim='time')

In [ ]:
plt.plot(spectra(ds, geometry='C')[-50:-10], '.-')
plt.plot(spectra(ds, geometry='B')[-50:-10], '.-')

In [ ]:
np.linalg.matrix_rank(ds.CLV[0])

In [ ]:
ds = xr.open_mfdataset(f'ginelli/trajectory/*.nc', parallel=True, combine='by_coords', concat_dim='time')

In [ ]:
ds

In [ ]:
h_experiment=[]
for i in tqdm(range(1, 6)):
    ds = xr.open_mfdataset(f'ginelli/step5/*.nc', parallel=True, combine='by_coords', concat_dim='time')
    h_experiment.append(ds)

In [ ]:
cupboard = 'ginelli/step5'
name = LyapunovLooker.name

save = cupboard + f'/{name}' + f'{1}.nc'
print(save)

In [ ]:
LyapunovLooker.observations.to_netcdf(save)

In [ ]:
ds.close()

In [ ]:
# ----------------------------------------
# TrajectoryObserver
# ----------------------------------------

class TrajectoryObserver:
    """Observes the Q Matrix in the forward Ginelli Steps."""

    def __init__(self, ginelli):
        """param, ginelli: Ginelli Forward Stepper being obseved."""
        
        self.name = 'Trajectory'
        self.dump_count = 0

        # Knowledge from Ginelli object
        self.integrator = ginelli.integrator # stepper we're associated with
        
        # Needed knowledge of the integrator
        self._K = self.integrator.K
        self._J = self.integrator.J
        self._parameters = self.integrator.parameter_dict
        self.scale = self.integrator.b

        # Trajectory Observation logs
        self.time_obs = [] # Times we've made observations
        self.x_obs = []
        self.y_obs = []
        self.dx_obs = []
        self.dy_obs = []

    def look(self, ginelli):
        """Observes trajectory of L96 trajectory"""
        
        integrator = ginelli.integrator

        # Note the time
        self.time_obs.append(integrator.time)

        # Making Observations
        self.x_obs.append(integrator.X.copy())
        self.y_obs.append(integrator.Y.copy()/self.scale) # Integrator solves transformed equations
        self.dx_obs.append(integrator.dx.copy())
        self.dy_obs.append(integrator.dy.copy()/self.scale)

    @property
    def observations(self):
        """cupboard: Directory where to write netcdf."""
        if (len(self.x_obs) == 0):
            print('I have no observations! :(')
            return

        dic = {}
        _time = self.time_obs
        dic['X'] = xr.DataArray(self.x_obs, dims=['time', 'K'], name='X',
                                coords = {'time': _time,'K': np.arange(1, 1 + self._K)})
        dic['Y'] = xr.DataArray(self.y_obs, dims=['time', 'KJ'], name='Y',
                                coords = {'time': _time, 'KJ': np.arange(1, 1 + self._K * self._J)})
        dic['dx'] = xr.DataArray(self.x_obs, dims=['time', 'K'], name='dx',
                                coords = {'time': _time,'K': np.arange(1, self._K + 1)})
        dic['dy'] = xr.DataArray(self.y_obs, dims=['time', 'KJ'], name='dy',
                                coords = {'time': _time, 'KJ': np.arange(1, 1 + self._K * self._J)})

        # Slow Variables above fast ones
        dic['X_repeat'] = xr.DataArray(np.repeat(self.x_obs, self._J, axis=1),
                                   dims=['time', 'KJ'], name='X_repeat',
                                    coords = {'time': _time,'KJ': np.arange(1, 1 + self._K * self._J)})# X's above the y's

        return xr.Dataset(dic, attrs= self._parameters)

    def wipe(self):
        """Erases observations"""
        self.time_obs = []
        self.x_obs = []
        self.y_obs = []
        self.dx_obs = []
        self.dy_obs = []

    def dump(self, cupboard, name=None):
        """ Saves observations to netcdf and wipes.
        cupboard: Directory where to write netcdf.
        name: file name"""

        if (len(self.x_obs) == 0):
            print('I have no observations! :(')
            return

        if name == None:
            name=self.name

        save = cupboard + f'/{name}' + f'{self.dump_count + 1}.nc'
        self.observations.to_netcdf(save)
        print(f'Observations written to {save}. Erasing personal log.\n')
        self.wipe()
        self.dump_count +=1


In [ ]:
runn